In [61]:
# Note: you may need to run the following in administrator mode in Windows, esp if you had to install spacy's language models
# as admin.  See the README for details.
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
import spacy
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

import praw
import re
import random

In [67]:
# Initialize key variables here
credential_file = "credentials.key"
targetuser = "ryan_holiday" #User you are targetting, NOT necessarily your username (unless you want to do this on yourself)
test_portion = 0.1 #What fraction of the user's posts and submissions to use as the test set?
bs = 64
bptt = 70

In [30]:
try:
    with open(credential_file, 'r') as f:
        creds = f.read().split('\n')
    personal = creds[0]
    secret = creds[1]
    username = creds[2]
    password = creds[3]
except OSError as e:
    print("You didn't create a credential file! Please see sample_credentials.key")
    print("Then go to http://www.storybench.org/how-to-scrape-reddit-with-python/")
    print("And register a new app named fastai_reddit in your reddit account.")
    print("And insert the values into sample_credentials.key and save it as {}.".format(credential_file))
    raise(e)

In [31]:
reddit = praw.Reddit(client_id=personal, client_secret=secret, user_agent='fastai_reddit', username=username, \
                     password=password)

In [51]:
def noquotes(text):
    return re.sub(">.+?(\n|$)","",text)

In [32]:
# Test that our Reddit connection works
subreddit = reddit.subreddit('Nootropics')
test = list(subreddit.top(limit=10))[0]
print(test.title)

We need to stop this.


In [52]:
# Test getting the latest 2 comments from the user
# See: https://www.reddit.com/r/redditdev/comments/5gt42t/praw_getting_users_last_1000_comments_causing/ 
user = reddit.redditor(targetuser)
for comment in user.comments.new(limit=2):
    print(noquotes(comment.body))

Read it however you like my friend. It's designed to be one page a day (and tied to the day's date) but it will work in just about any form. 
If you fear getting caught doing something, it's probably a sign you shouldn't be doing it. 


In [53]:
# Test getting the latest 2 submissions from the user
# See: https://www.reddit.com/r/redditdev/comments/38wzzm/praw_how_to_get_submission_text/
# TODO: write a function to remove quoted text between a ">" and a new line
for sub in user.submissions.new(limit=2):
    print(noquotes(sub.selftext))

I've just finished reading Thirteen Days as well as a number of interesting articles that attempt to "correct the record" about the Cuban Missile Crisis (including [this one from The Atlantic](https://www.theatlantic.com/magazine/archive/2013/01/the-real-cuban-missile-crisis/309190/?single_page=true)). Here is where I am confused. In Thirteen Days, RFK makes the case that Kennedy had never really wanted the Jupiter missiles in Turkey, that he had repeatedly asked for them to be removed, and possibly even thought they already had been. Therefore he was frustrated when Khrushchev demanded their removal because it was clear that the US's hardline position re: Cuba was less compelling with their existence. (Ultimately he ended up trading away their removal).

However, in a lot of contrarian histories I've read blame Kennedy for starting the crisis by placing the missiles there in 1961 (even though plans appear to predate his presidency) and more or less put the responsibility on him for pu

In [54]:
text = []
for comment in user.comments.new(limit=None):
    text.append(noquotes(comment.body))


In [55]:
print(len(text))

1000


In [56]:
subs = []
for sub in user.submissions.new(limit=None):
    newsub = noquotes(sub.selftext)
    if len(newsub) > 0:
        subs.append(newsub)

In [57]:
len(subs)

60

In [66]:
alltext = text + subs
random.shuffle(alltext)
split = round(test_portion*len(alltext))
train = pd.DataFrame()
test = pd.DataFrame()
test['text'] = alltext[0:split]
train['text'] = alltext[split:]

In [68]:
TEXTS = data.Field(lower=True, tokenize="spacy")

In [69]:
md = LanguageModelData.from_dataframes("./", TEXTS, 'text',train,test,test, bs=bs, bptt=bptt, min_freq=10)